In [1]:
#!pip install transformers --upgrade
# !pip install datasets
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install sklearn
#!pip install scikit-learn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
from transformers import BertTokenizer,BertModel,BertConfig
from transformers import ErnieModel
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import logging
import pandas as pd
import numpy as np

ImportError: cannot import name 'ErnieModel' from 'transformers' (/opt/conda/lib/python3.7/site-packages/transformers/__init__.py)

In [ ]:
tag=4
num_classes=2

In [ ]:
df = pd.read_csv('/kaggle/input/edos-dev-task/training.csv',engine='python',names=['rewire_id','text','label_sexist','label_category','label_vector','label_sexist0','sentiment'])
df.info()

In [ ]:
df['label_vector'].value_counts()

In [ ]:
df.loc[df['label_vector']=='1.1 threats of harm','c_1']=0
df.loc[df['label_vector']=='1.2 incitement and encouragement of harm','c_1']=1
#df.loc[df['label_vector']=='none','c_1']=100

df.loc[df['label_vector']=='2.1 descriptive attacks','c_2']=0 #717 2.2 0.176
df.loc[df['label_vector']=='2.2 aggressive and emotive attacks','c_2']=1 #673 2.2 0.176
df.loc[df['label_vector']=='2.3 dehumanising attacks & overt sexual objectification','c_2']=2 # 200 8 0.64
#df.loc[df['label_vector']=='none','c_2']=100

df.loc[df['label_vector']=='3.1 casual use of gendered slurs, profanities, and insults','c_3']=0
df.loc[df['label_vector']=='3.2 immutable gender differences and gender stereotypes','c_3']=1
df.loc[df['label_vector']=='3.3 backhanded gendered compliments','c_3']=2
df.loc[df['label_vector']=='3.4 condescending explanations or unwelcome advice','c_3']=3
# 637,417,64,47
0.1,0.15,0.35, 0.4

#df.loc[df['label_vector']=='none','c_3']=100

df.loc[df['label_vector']=='4.1 supporting mistreatment of individual women','c_4']=0
df.loc[df['label_vector']=='4.2 supporting systemic discrimination against women as a group','c_4']=1
#df.loc[df['label_vector']=='none','c_4']=100
# 75 258
# 0.22 0.78

df.fillna(value=100)

df.info()
#print(df)
#df.to_csv("/kaggle/working/testfile_see2.csv",index=False) #save to file


In [ ]:
##设置当前的任务为：1 2 3 4 
##分别对应 7 8 9 10
print(df['c_1'].value_counts())
df['c_2'].value_counts()
df['c_3'].value_counts()

In [ ]:
df2 = df.fillna(
    100,  # nan的替换值
    inplace=False  # 是否跟换源文件
)

file1=np.array(df2)

data=[]
for item in file1:
    if item[tag+6]!=100:
        item[tag+6]=item[tag+6]#label
        item[1]=str(item[1])#text
        item[0]=item[0][19:]
        #print(item[1],item[7],item[0])
        data.append(list(item))

x=[x[1] for x in data]#text
y=[x[tag+6] for x in data]#label
z=[x[0] for x in data]#id


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, z_train, z_test =  train_test_split(x, y, z, test_size=0.1,train_size=0.9)
#z_test

In [ ]:
df_dev=pd.read_csv('/kaggle/input/edos-dev-task/dev_task_c_entries.csv')
df_dev.info()

df_re=pd.read_csv('/kaggle/input/edosbtest/testfile_large_63.5.csv')
df_re.info()
df_data=pd.merge(df_dev,df_re,on='rewire_id')
df_data.info()
x_dev= df_data['text'].tolist()

df_data2=np.array(df_data)
#z_dev=[int(x[19:]) for x in list(df_dev['rewire_id']) if x[2][0]==tag]
#z_dev=[x[0] for x in list(df_data['label_pred']) if int(x[0])==tag]
z_dev=[x[0][19:] for x in df_data2 if int(x[2][0])==tag]
print(z_dev)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("nghuyong/ernie-2.0-large-en")
train_encoding = tokenizer(x_train, truncation=True, padding=True, max_length=50)
test_encoding = tokenizer(x_test, truncation=True, padding=True, max_length=50)
dev_encoding=tokenizer(x_dev,truncation=True,padding=True, max_length=50)

In [ ]:
print(train_encoding.keys())

## 数据集封装和处理

In [ ]:
# 数据集读取, 继承torch的Dataset类，方便后面用DataLoader封装数据集
class NewsDataset(Dataset):
    def __init__(self, encodings, labels,ids):
        self.encodings = encodings
        self.labels = labels
        self.ids=ids
    
    #这里的idx是为了让后面的DataLoader成批处理成迭代器，按idx映射到对应数据
    def __getitem__(self, idx):
        item = {key: torch.as_tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.as_tensor(int(self.labels[idx]))
        item['ids']=torch.as_tensor(int(self.ids[idx]))
        return item
    
    #数据集长度。通过len(这个实例对象)，可以查看长度
    def __len__(self):
        return len(self.labels)
    
class DevDataset(Dataset):
    def __init__(self, encodings,ids):
        self.encodings = encodings
        self.ids=ids
    
    #这里的idx是为了让后面的DataLoader成批处理成迭代器，按idx映射到对应数据
    def __getitem__(self, idx):
        item = {key: torch.as_tensor(val[idx]) for key,val in self.encodings.items()}
         #item['encodings']=torch.as_tensor(self.encodings[idx])
        item['ids']=torch.as_tensor(int(self.ids[idx]))
        return item
    
    #数据集长度。通过len(这个实例对象)，可以查看长度
    def __len__(self):
        return len(self.ids)
    
#将数据集包装成torch的Dataset形式
train_dataset = NewsDataset(train_encoding, y_train,z_train)
test_dataset = NewsDataset(test_encoding, y_test,z_test)
dev_dataset=DevDataset(dev_encoding,z_dev)
# 单个读取到批量读取
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)
dev_dataloader=DataLoader(dev_dataset,batch_size=16,shuffle=True)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self,num_classes,checkpoint="bert-large-uncased"):
        super().__init__()
        self.pretrained = ErnieModel.from_pretrained("nghuyong/ernie-2.0-large-en")
        self.fc = torch.nn.Sequential(torch.nn.Linear(1024, num_classes))
 
    def forward(self, input_ids, attention_mask):
        logits = self.pretrained(input_ids=input_ids, attention_mask=attention_mask)
        logits = logits.last_hidden_state[:, 0]
        logits = self.fc(logits)
        return logits
    
# class Config(object):

#     def __init__(self):
#         self.pre_bert_path="nghuyong/ernie-1.0"
#         self.train_path = 'data/dataset_train.csv'  # 训练集
#         self.dev_path = 'data/dataset_valid.csv'  # 验证集
#         self.test_path = 'data/test.csv'  # 测试集
#         self.class_path = 'data/class.json'  # 类别名单
#         self.save_path ='mymodel/ernie.pth'        # 模型训练结果
#         self.num_classes=10
#         self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # 设备

#         self.epochs = 10  # epoch数
#         self.batch_size = 128  # mini-batch大小
#         self.maxlen = 32  # 每句话处理成的长度(短填长切)
#         self.learning_rate = 5e-4                                       # 学习率
#         self.hidden_size=768
#         self.tokenizer = AutoTokenizer.from_pretrained(self.pre_bert_path)

# class Model(nn.Module):
#     def __init__(self, config):
#         super(Model, self).__init__()
#         self.ernie=AutoModel.from_pretrained(config.pre_bert_path)
#         #设置不更新预训练模型的参数
#         for param in self.ernie.parameters():
#             param.requires_grad = False
#         self.fc = nn.Linear(config.hidden_size, config.num_classes)
#     def forward(self, input):
#         out=self.ernie(input_ids =input['input_ids'],attention_mask=input['attention_mask'],token_type_ids=input['token_type_ids'])
#         #只取最后一层CLS对应的输出
#         out = self.fc(out.pooler_output)
#         return out

## 实例化模型、定义损失函数和优化器

In [ ]:
from torch.autograd import Variable
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device,'能用')

model = Model(num_classes).to(device)

class FocalLoss(nn.Module):
    #def __init__(self, class_num, alpha=[0.78,0.2], gamma=2, use_alpha=True, size_average=True):#0.6+
    #def __init__(self, class_num, alpha=[0.18,0.22,0.7], gamma=2, use_alpha=True, size_average=True):#0.609
    #def __init__(self, class_num, alpha=[0.08,0.13,0.37,0.42], gamma=2, use_alpha=True, size_average=True):#0.88
    def __init__(self, class_num, alpha=[0.74,0.26], gamma=2, use_alpha=True, size_average=True):#0.91 0.76


        super(FocalLoss, self).__init__()
        self.class_num = class_num
        self.alpha = alpha
        self.gamma = gamma
        if use_alpha:
            self.alpha = torch.tensor(alpha).cuda()
        self.softmax = nn.Softmax(dim=1)
        self.use_alpha = use_alpha
        self.size_average = size_average
    def forward(self, pred, target):
        prob = self.softmax(pred.view(-1,self.class_num))
        prob = prob.clamp(min=0.0001,max=1.0)
        target_ = torch.zeros(target.size(0),self.class_num).cuda()
        target_.scatter_(1, target.view(-1, 1).long(), 1.)
        if self.use_alpha:
            batch_loss = - self.alpha.double() * torch.pow(1-prob,self.gamma).double() * prob.log().double() * target_.double()
        else:
            batch_loss = - torch.pow(1-prob,self.gamma).double() * prob.log().double() * target_.double()
        batch_loss = batch_loss.sum(dim=1)
        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss

criterion = FocalLoss(class_num=num_classes)
    
# 优化方法
#过滤掉被冻结的参数，反向传播需要更新的参数
optim = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
total_steps = len(train_loader) * 1
scheduler = get_linear_schedule_with_warmup(optim, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## 定义训练函数

In [ ]:
from statistics import mean
def calculate_f1(stacked,num_classes):
    stacked=torch.as_tensor(stacked,dtype=torch.int64)
    cmt=torch.zeros(num_classes,num_classes,dtype=torch.int32)
    #cmt
    for p in stacked:
        tl,pl=p.tolist()
        cmt[tl,pl]=cmt[tl,pl]+1
    M=cmt
    n = len(M)
    pre_all=[]
    rec_all=[]
    f1_all=[]
    sum_acc=0
    for i in range(len(M[0])):
        rowsum, colsum = sum(M[i]), sum(M[r][i] for r in range(n))
        try:
            precision=float(M[i][i]/float(colsum))
            pre_all.append(precision)
            print(float(M[i][i]))
            print(float(colsum))
            recall=float(M[i][i]/float(rowsum))
            rec_all.append(recall)
            f1=2*(precision*recall)/(precision+recall)
            f1_all.append(f1)

            sum_acc+=M[i][i].item()
        except ZeroDivisionError:
            print('precision: %s' % 0, 'recall: %s' % 0)
    print(cmt)
    accuracy=sum_acc/((sum(M).sum()).item())
    print('accuracy:%s' % accuracy ,'precision: %s' % (mean(pre_all)), 'recall: %s' % (mean(rec_all)),'F1: %s ' % (mean(f1_all)))

In [ ]:
import numpy as np
import pandas as pd

def save_to_csv(stacked,tag):
    stacked=torch.as_tensor(stacked).cpu()
    t_np = stacked.numpy() #convert to Numpy array
    df = pd.DataFrame(t_np) #convert to a dataframe
    
    df.columns=['rewire_id', 'label_pred']
    df['rewire_id']=['sexism2022_english-'+str(x) for x in df['rewire_id']]
    if tag==1:
        df.loc[df['label_pred']==0,'label_pred']='1.1 threats of harm'
        df.loc[df['label_pred']==1,'label_pred']='1.2 incitement and encouragement of harm'
        print(df)
        df.to_csv("/kaggle/working/testfile_c_1.csv",index=False) #save to file
    elif tag==2:
        df.loc[df['label_pred']==0,'label_pred']='2.1 descriptive attacks'
        df.loc[df['label_pred']==1,'label_pred']='2.2 aggressive and emotive attacks'
        df.loc[df['label_pred']==2,'label_pred']='2.3 dehumanising attacks & overt sexual objectification'
        df.to_csv("/kaggle/working/testfile_c_2.csv",index=False) #save to file
    elif tag==3:
        df.loc[df['label_pred']==0,'label_pred']='3.1 casual use of gendered slurs, profanities, and insults'
        df.loc[df['label_pred']==1,'label_pred']='3.2 immutable gender differences and gender stereotypes'
        df.loc[df['label_pred']==2,'label_pred']='3.3 backhanded gendered compliments'
        df.loc[df['label_pred']==3,'label_pred']='3.4 condescending explanations or unwelcome advice'
        df.to_csv("/kaggle/working/testfile_c_3.csv",index=False) #save to file
    elif tag==4:
        df.loc[df['label_pred']==0,'label_pred']='4.1 supporting mistreatment of individual women'
        df.loc[df['label_pred']==1,'label_pred']='4.2 supporting systemic discrimination against women as a group'
        df.to_csv("/kaggle/working/testfile_c_4.csv",index=False) #save to file

    
    print("保存成功！")
    #Then, to reload:

In [ ]:
h = pd.read_csv("/kaggle/working/testfile_c_1.csv")
for i in range(2,5):
    print(i)
    h1 = pd.read_csv("/kaggle/working/testfile_c_{}.csv".format(i))
    h= pd.concat([h,h1],axis=0)
    print(len(h))
h.to_csv("/kaggle/working/testfile_c_all.csv",index=False) #save to file

In [ ]:
# 精度计算
#imports untils
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
batch_size=16
def validation():
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    with torch.no_grad():
        results=torch.zeros(0).to(device)
        label_all=torch.zeros(0).to(device)
        id_all=torch.zeros(0).to(device)
        for batch in test_dataloader:
            # 正常传播
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            ids=batch['ids'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            total_eval_loss += loss.item()
            logits = outputs
            
            # 获取预测结果
            preds = outputs.argmax(1).to(device)
            results=torch.cat((results,preds),dim=0)#预测的结果
            
            label_all=torch.cat((label_all,labels),dim=0)
            id_all=torch.cat((id_all,ids),dim=0)
            
        stacked=torch.stack(
                (
                    torch.as_tensor(label_all).to(device),
                    torch.as_tensor(list(map(int,results))).to(device)
                )
            )
        stacked=stacked.t()
        calculate_f1(stacked,num_classes)
        
#         stacked_saved=torch.stack(
#                     (
#                         torch.as_tensor(list(map(int,id_all))).to(device),
#                         torch.as_tensor(list(map(int,label_all))).to(device),#真实结果
#                         torch.as_tensor(list(map(int,results))).to(device)#预测结果
#                     )
#                 )
#         stacked_saved=stacked_saved.t()


        #print(stacked_saved)
    print("Average testing loss: %.4f"%(total_eval_loss/len(test_dataloader)))
    print("-------------------------------")

In [ ]:
def train(tag):
    batch_size=16
    for epoch in range(2):
        print("------------Epoch: %d ----------------" % epoch)
        train_loss = 0.0 # 训练损失
        val_loss = 0.0 # 验证损失
        model.train() # 声明开始训练
        total_train_loss = 0
        iter_num = 0
        total_iter = len(train_loader)
        
        results=torch.zeros(0).to(device)
        label_all=torch.zeros(0).to(device)
        labels=torch.zeros(0).to(device)
        for batch in train_loader:
            # 正向传播
            optim.zero_grad()
            #print(batch)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            ids=batch['ids'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)

            loss = criterion(outputs, labels)    

            total_train_loss += loss.item()

            loss.backward()# 反向传播
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #梯度裁剪，防止梯度爆炸  
            
            # 获取预测结果
            preds = outputs.argmax(1).to(device)
            
            
            stacked=torch.stack(
                    (
                        ids,
                        labels,
                        preds
                    )
                )
            stacked=stacked.t()
            

            # 参数更新
            optim.step()
            scheduler.step()

            iter_num += 1
            if(iter_num % 100==0):
                print("epoth: %d, iter_num: %d, loss: %.4f, %.2f%%" % (epoch, iter_num, loss.item(), iter_num/total_iter*100))
        print("Epoch: %d, Average training loss: %.4f"%(epoch, total_train_loss/len(train_loader)))     
                
        validation()
        
        model.eval()
        print("--------------开始分类dev集-----------------")
        with torch.no_grad():
            results=torch.zeros(0).to(device)
            #label_all=torch.zeros(0).to(device)
            ids_all=torch.zeros(0).to(device)
            for batch in dev_dataloader:
                # 正常传播
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                #labels = batch['labels'].to(device)
                ids=batch['ids'].to(device)
                print(ids)

                outputs = model(input_ids, attention_mask=attention_mask)
                

                # 获取预测结果
                preds = outputs.argmax(1).to(device)
                results=torch.cat((results,preds),dim=0)#预测的结果
                ids_all=torch.cat((ids_all,ids),dim=0)

            stacked_saved=torch.stack(
                        (
                            torch.as_tensor(list(map(int,ids_all))).to(device),
                            torch.as_tensor(list(map(int,results))).to(device)#预测结果
                        )
                    )
            print(ids_all)
            stacked_saved=stacked_saved.t()
            save_to_csv(stacked_saved,tag)

        print("--------------dev完成-----------------")

In [ ]:
train(tag)